# 核岭回归课堂演讲稿（3部分，口述版）

**Kernel Ridge Regression Class Presentation (3 Sections, Full Script)**

---

## 第一部分：概述与原理

**Part 1: Overview & Principle**
（约4分钟）

**中文口述稿：**
大家好，今天我要和大家分享的是**核岭回归**，英文叫 **Kernel Ridge Regression**，简称 **KRR**。
在进入具体算法之前，我们先回顾两个核心背景：岭回归和核方法。

首先是**岭回归**。
岭回归是在线性回归的基础上加入了一个L2正则化项，这个项的作用就是惩罚模型的参数过大，从而减少过拟合风险。它的损失函数形式是：

$$
\min_w \|y - Xw\|^2 + \lambda \|w\|^2
$$

其中 $\lambda$ 就是正则化参数，它越大，惩罚力度越强，模型越保守。

接下来是**核方法**。
在现实问题中，很多数据关系并不是线性的，比如正弦曲线、圆形分布等。核方法的思想是：通过一个核函数，把原来的输入数据隐式地映射到一个高维空间，在这个空间里，原本复杂的非线性关系有可能变成线性关系。这样我们就可以用线性方法去解决非线性问题。
常见的核函数有：

* **线性核（Linear）**：不做映射，直接内积。
* **多项式核（Polynomial）**：用多项式来组合特征。
* **RBF核（Radial Basis Function）**：用高斯函数衡量相似度，常用于非线性拟合。
* **Sigmoid核**：类似神经元的激活函数。

**核岭回归**就是把岭回归和核方法结合起来，在高维空间做岭回归。它的目标函数是：

$$
\min_{\alpha} \| y - K \alpha \|^2 + \lambda \alpha^T K \alpha
$$

这里的 $K$ 是核矩阵，它的每个元素是两个样本之间的核函数值；$\alpha$ 是对偶系数；$\lambda$ 控制正则化强度。

这种方法有两个好处：

1. 能处理复杂的非线性关系。
2. 有正则化机制，能避免过拟合。

**English Script:**
Hello everyone, today I’ll talk about **Kernel Ridge Regression**, or **KRR**.
Before diving into the details, let’s review two important concepts: ridge regression and kernel methods.

First, **ridge regression** adds an L2 regularization term to the least squares loss. This penalizes large parameter values, which helps prevent overfitting. The loss function is:

$$
\min_w \|y - Xw\|^2 + \lambda \|w\|^2
$$

where $\lambda$ controls the strength of regularization.

Next, **kernel methods**. Many real-world relationships are nonlinear. Kernel methods map the input data into a high-dimensional space—implicitly—through a kernel function, where the data might become linearly separable. Common kernels include:

* **Linear**: inner product without mapping.
* **Polynomial**: polynomial combinations of features.
* **RBF**: Gaussian similarity, very powerful for nonlinear fitting.
* **Sigmoid**: similar to neural network activation functions.

KRR combines these two: ridge regression in a high-dimensional space defined by the kernel. The objective is:

$$
\min_{\alpha} \| y - K \alpha \|^2 + \lambda \alpha^T K \alpha
$$

where $K$ is the kernel matrix, $\alpha$ are the dual coefficients, and $\lambda$ is the regularization strength.
This gives us **nonlinear modeling power with regularization control**.

---


## 第二部分：实现与核心模块

**Part 2: Implementation & Core Modules**
（约4分钟）

---

#### **中文口述稿：**
接下来我们看看实现细节，我们的代码主要由三个文件组成。

第一个是 **`kernels.py`**，这里定义了四种核函数，我先给大家逐个介绍公式和作用：

1. **线性核（Linear Kernel）**

   $$
   K(x, x') = x^\top x'
   $$

   直接计算两个输入向量的内积，不进行非线性映射，适合线性关系数据。

2. **多项式核（Polynomial Kernel）**

   $$
   K(x, x') = \left( \gamma \, x^\top x' + c_0 \right)^d
   $$

   * $\gamma$：缩放参数（默认 $\gamma = \frac{1}{n_{\text{features}}}$）
   * $c_0$：偏置项
   * $d$：多项式阶数
     适合用多项式组合特征的场景，比如拟合曲线或表面。

3. **RBF 核（Radial Basis Function Kernel / Gaussian Kernel）**

   $$
   K(x, x') = \exp\left( -\gamma \, \|x - x'\|^2 \right)
   $$

   * $\gamma$ 控制高斯分布的宽度（默认 $\gamma = \frac{1}{n_{\text{features}}}$）
     RBF 核非常适合非线性问题，能捕捉复杂的局部模式。

4. **Sigmoid 核（Sigmoid Kernel）**

   $$
   K(x, x') = \tanh\left( \gamma \, x^\top x' + c_0 \right)
   $$

   * $\gamma$：缩放参数
   * $c_0$：偏置项
     Sigmoid 核与神经网络中的激活函数类似，在一定条件下能模拟两层神经网络。

---

第二个文件是 **`kernel_ridge.py`**，核心类是 `KernelRidge`。它有几个关键方法：

* `__init__`：初始化模型，指定核类型、正则化参数alpha、gamma、degree等。
* `fit(X, y)`：先计算核矩阵 $K$，然后解线性方程组 $(K + n\alpha I)\alpha = y$，得到对偶系数。
* `predict(X)`：计算测试点和训练点的核矩阵，然后与对偶系数相乘，得到预测结果。

---

第三个文件是 **`example_usage.py`**，这是一个示例文件。
这里我们首先生成一批非线性数据，比如：

$$
y = \sin(x) + \text{噪声}
$$

然后用RBF核训练模型，画出预测曲线。同时我们也比较了不同核函数的预测效果，比如线性核就很难拟合正弦曲线，而RBF核表现得非常好。

---

#### **English Script:**
Let’s look at the implementation. Our code has three main files.

The first is **`kernels.py`**, defining four kernels, with their formulas:

1. **Linear Kernel**

   $$
   K(x, x') = x^\top x'
   $$

   Direct dot product, suitable for linear data.

2. **Polynomial Kernel**

   $$
   K(x, x') = \left( \gamma \, x^\top x' + c_0 \right)^d
   $$

   * $\gamma$: scaling factor (default $\gamma = \frac{1}{n_{\text{features}}}$)
   * $c_0$: bias term
   * $d$: degree
     Suitable for polynomial feature mapping.

3. **RBF Kernel**

   $$
   K(x, x') = \exp\left( -\gamma \, \|x - x'\|^2 \right)
   $$

   * $\gamma$ controls Gaussian width
     Great for nonlinear problems and local patterns.

4. **Sigmoid Kernel**

   $$
   K(x, x') = \tanh\left( \gamma \, x^\top x' + c_0 \right)
   $$

   * $\gamma$: scaling factor
   * $c_0$: bias term
     Similar to neural network activation.

---

The second file is **`kernel_ridge.py`**, core class `KernelRidge`:

* `__init__`: initializes kernel type, alpha, gamma, degree, etc.
* `fit(X, y)`: computes $K$, solves $(K + n\alpha I)\alpha = y$ for dual coefficients.
* `predict(X)`: computes kernel matrix between test and train data, multiplies by dual coefficients for predictions.

---

The third file is **`example_usage.py`**, a demo:
Generate nonlinear data:

$$
y = \sin(x) + \text{noise}
$$

Train with RBF kernel, plot results, and compare kernels. Linear fails on sine curves, RBF does well.

---


## 第三部分：实验与总结

**Part 3: Experiments & Conclusion**
（约4分钟）

**中文口述稿：**
在实验部分，我们的 `example_usage.py` 展示了完整过程。

首先是数据生成：我们创建了1000个点，x在0到20之间，y是正弦函数加上随机噪声。这样数据的非线性特征非常明显。

然后，我们用RBF核进行训练。RBF核有一个超参数gamma，它控制高斯分布的宽度。gamma太大，模型容易过拟合；太小，模型会欠拟合。在实验中我们选取了0.5作为gamma。

训练后，我们画出预测曲线，可以看到红色的预测线几乎和绿色的真实曲线重合，这说明模型拟合得非常好。

我们还比较了不同核的表现：

* 线性核：只能拟合大致趋势，无法捕捉正弦的波动。
* 多项式核：拟合能力比线性好，但容易在波峰波谷处出现偏差。
* RBF核：效果最佳，几乎重合。
* Sigmoid核：在某些段落表现不错，但整体拟合不如RBF。

最后，总结一下核岭回归的优缺点：
优点：

1. 能处理非线性数据。
2. 正则化抑制过拟合。
   缺点：
3. 样本数很大时，计算核矩阵会很耗内存。
4. 参数选择（如gamma、alpha）需要交叉验证。

**English Script:**
In the experiments, `example_usage.py` shows the full pipeline.

We generate 1000 points, x from 0 to 20, y as a sine function plus random noise—making the nonlinearity obvious.

We train with the RBF kernel. The gamma parameter controls the Gaussian width: too large risks overfitting, too small underfits. We use gamma = 0.5.

After training, the predicted curve almost overlaps with the true curve—showing strong fitting ability.

Kernel comparison results:

* Linear: captures only the trend, misses sine oscillations.
* Polynomial: better than linear but deviates at peaks and troughs.
* RBF: best performance, near-perfect fit.
* Sigmoid: good in some segments but not overall.

Summary of pros and cons:
**Pros**:

1. Handles nonlinear data.
2. Regularization prevents overfitting.
   **Cons**:
3. Computing kernel matrix is costly for large datasets.
4. Parameter tuning (gamma, alpha) requires cross-validation.

---

我可以帮你把这份**12分钟口述稿配合你的代码绘图做成PPT**，这样每部分讲的时候有可视化曲线对照，课堂效果会更好。
你要我帮你做这个PPT版本吗？这样讲起来会更直观。
